## Day 7 Checkpoint 1: Database Models & CRUD Test

<img style="float: right;" src="../img/logo.png" width="120"><br>

<div style="text-align: right"> <b>Research Curator Team</b></div>
<div style="text-align: right"> Initial issue : 2025.12.04 </div>
<div style="text-align: right"> last update : 2025.12.04 </div>

개정 이력  
- `2025.12.04` : Database models & CRUD operations test

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv

load_dotenv()

### 1. Database Connection Test

In [ ]:
from app.core.config import settings
from app.db.session import SessionLocal, engine

print(f"Database URL: {settings.database_url_str}")
print(f"Engine: {engine}")

# Test connection
try:
    with engine.connect() as conn:
        print("✅ Database connection successful!")
except Exception as e:
    print(f"❌ Database connection failed: {e}")

### 2. Apply Alembic Migration

Run in terminal:
```bash
alembic upgrade head
```

### 3. User CRUD Operations Test

In [ ]:
from app.db import crud
from app.db.session import SessionLocal

db = SessionLocal()

# Create a test user
test_user = crud.create_user(
    db=db,
    email="test@example.com",
    name="Test User"
)

print(f"Created user: {test_user}")
print(f"User ID: {test_user.id}")
print(f"Email: {test_user.email}")
print(f"Name: {test_user.name}")

user_id = test_user.id

In [ ]:
# Get user by ID
user = crud.get_user_by_id(db, user_id)
print(f"Retrieved user by ID: {user}")

# Get user by email
user = crud.get_user_by_email(db, "test@example.com")
print(f"Retrieved user by email: {user}")

In [ ]:
# Update user
updated_user = crud.update_user(db, user_id, name="Updated Test User")
print(f"Updated user: {updated_user}")
print(f"New name: {updated_user.name}")

### 4. UserPreference CRUD Operations Test

In [ ]:
# Create user preferences
preference = crud.create_user_preference(
    db=db,
    user_id=user_id,
    research_fields=["Machine Learning", "NLP"],
    keywords=["transformers", "attention", "LLM"],
    sources={"arxiv": True, "google_scholar": True},
    info_types={"paper": 50, "news": 30, "report": 20},
    email_time="09:00",
    daily_limit=10,
)

print(f"Created preference: {preference}")
print(f"Research fields: {preference.research_fields}")
print(f"Keywords: {preference.keywords}")
print(f"Email time: {preference.email_time}")

In [ ]:
# Get user preferences
pref = crud.get_user_preference(db, user_id)
print(f"Retrieved preference: {pref}")
print(f"Info types: {pref.info_types}")

In [ ]:
# Update preferences
updated_pref = crud.update_user_preference(
    db,
    user_id,
    keywords=["transformers", "attention", "LLM", "GPT"],
    daily_limit=15,
)
print(f"Updated preference: {updated_pref}")
print(f"New keywords: {updated_pref.keywords}")
print(f"New daily limit: {updated_pref.daily_limit}")

### 5. CollectedArticle CRUD Operations Test

In [ ]:
from datetime import datetime, UTC

# Create test articles
article1 = crud.create_article(
    db=db,
    title="Attention Is All You Need",
    content="Full paper content here...",
    summary="혁신적인 Transformer 아키텍처를 소개하는 논문입니다.",
    source_url="https://arxiv.org/abs/1706.03762",
    source_type="paper",
    category="Deep Learning",
    importance_score=0.95,
    article_metadata={
        "authors": ["Vaswani et al."],
        "citations": 90000,
    },
    published_at=datetime.now(UTC),
)

article2 = crud.create_article(
    db=db,
    title="GPT-4 Technical Report",
    content="Technical report content...",
    summary="OpenAI의 최신 대규모 언어 모델에 대한 기술 리포트입니다.",
    source_url="https://arxiv.org/abs/2303.08774",
    source_type="paper",
    category="LLM",
    importance_score=0.98,
    article_metadata={
        "authors": ["OpenAI"],
        "citations": 5000,
    },
    published_at=datetime.now(UTC),
)

article3 = crud.create_article(
    db=db,
    title="AI News: New Breakthrough in Reinforcement Learning",
    content="News article content...",
    summary="강화학습 분야의 새로운 돌파구에 대한 뉴스입니다.",
    source_url="https://techcrunch.com/ai-breakthrough",
    source_type="news",
    category="Reinforcement Learning",
    importance_score=0.75,
)

print(f"Created article 1: {article1.title} (score: {article1.importance_score})")
print(f"Created article 2: {article2.title} (score: {article2.importance_score})")
print(f"Created article 3: {article3.title} (score: {article3.importance_score})")

article_id = article1.id

In [ ]:
# Get article by ID
article = crud.get_article_by_id(db, article_id)
print(f"Retrieved article: {article.title}")

# Get article by URL
article = crud.get_article_by_url(db, "https://arxiv.org/abs/1706.03762")
print(f"Retrieved article by URL: {article.title}")

In [ ]:
# List all articles
all_articles = crud.list_articles(db, limit=10)
print(f"\nTotal articles: {len(all_articles)}")
for a in all_articles:
    print(f"- {a.title} ({a.source_type}, score: {a.importance_score})")

In [ ]:
# Filter by source type
papers = crud.list_articles(db, source_type="paper")
print(f"\nPapers only: {len(papers)}")
for p in papers:
    print(f"- {p.title}")

# Get top articles by importance
top_articles = crud.get_top_articles_by_importance(db, limit=5)
print(f"\nTop 5 articles by importance:")
for a in top_articles:
    print(f"- {a.title} (score: {a.importance_score})")

In [ ]:
# Count articles
total = crud.count_articles(db)
paper_count = crud.count_articles(db, source_type="paper")
news_count = crud.count_articles(db, source_type="news")

print(f"Total articles: {total}")
print(f"Papers: {paper_count}")
print(f"News: {news_count}")

### 6. SentDigest CRUD Operations Test

In [ ]:
# Create a digest
digest = crud.create_digest(
    db=db,
    user_id=user_id,
    article_ids=[str(article1.id), str(article2.id)],
)

print(f"Created digest: {digest}")
print(f"Article IDs: {digest.article_ids}")
print(f"Sent at: {digest.sent_at}")
print(f"Opened: {digest.email_opened}")

digest_id = digest.id

In [ ]:
# Mark digest as opened
opened_digest = crud.update_digest_opened(db, digest_id, datetime.now(UTC))
print(f"Digest opened: {opened_digest.email_opened}")
print(f"Opened at: {opened_digest.opened_at}")

In [ ]:
# List user digests
user_digests = crud.list_user_digests(db, user_id)
print(f"\nUser digests: {len(user_digests)}")
for d in user_digests:
    print(f"- Sent: {d.sent_at}, Opened: {d.email_opened}")

# Get latest digest
latest = crud.get_latest_digest(db, user_id)
print(f"\nLatest digest: {latest.sent_at}")

### 7. Feedback CRUD Operations Test

In [ ]:
# Create feedback
feedback1 = crud.create_feedback(
    db=db,
    user_id=user_id,
    article_id=article1.id,
    rating=5,
    comment="Excellent paper! Very insightful.",
)

feedback2 = crud.create_feedback(
    db=db,
    user_id=user_id,
    article_id=article2.id,
    rating=4,
    comment="Good technical report.",
)

print(f"Created feedback 1: {feedback1}")
print(f"Created feedback 2: {feedback2}")

In [ ]:
# Get user feedback for article
user_feedback = crud.get_user_feedback_for_article(db, user_id, article1.id)
print(f"User feedback for article: {user_feedback}")
print(f"Rating: {user_feedback.rating}")
print(f"Comment: {user_feedback.comment}")

In [ ]:
# List article feedbacks
article_feedbacks = crud.list_article_feedbacks(db, article1.id)
print(f"\nFeedbacks for article: {len(article_feedbacks)}")
for f in article_feedbacks:
    print(f"- Rating: {f.rating}, Comment: {f.comment}")

# List user feedbacks
user_feedbacks = crud.list_user_feedbacks(db, user_id)
print(f"\nUser's feedbacks: {len(user_feedbacks)}")
for f in user_feedbacks:
    print(f"- Rating: {f.rating}")

In [ ]:
# Get average rating
avg_rating = crud.get_article_average_rating(db, article1.id)
print(f"Average rating for article 1: {avg_rating}")

### 8. Test Relationships

In [ ]:
# Test User -> Preference relationship
user = crud.get_user_by_id(db, user_id)
print(f"User: {user.email}")
print(f"User's preference: {user.preference}")
print(f"Research fields: {user.preference.research_fields}")

# Test User -> Digests relationship
print(f"\nUser's digests: {len(user.digests)}")
for d in user.digests:
    print(f"- Digest sent at: {d.sent_at}")

# Test User -> Feedbacks relationship
print(f"\nUser's feedbacks: {len(user.feedbacks)}")
for f in user.feedbacks:
    print(f"- Feedback rating: {f.rating}")

# Test Article -> Feedbacks relationship
article = crud.get_article_by_id(db, article1.id)
print(f"\nArticle: {article.title}")
print(f"Article's feedbacks: {len(article.feedbacks)}")
for f in article.feedbacks:
    print(f"- Rating: {f.rating}, Comment: {f.comment}")

### 9. Cleanup (Optional)

In [ ]:
# Uncomment to delete test data
# crud.delete_user(db, user_id)  # This will cascade delete all related data
# crud.delete_article(db, article1.id)
# crud.delete_article(db, article2.id)
# crud.delete_article(db, article3.id)
# print("Test data deleted")

# Close database session
db.close()
print("Database session closed")

### Summary

✅ **Checkpoint 1 완료!**

테스트 완료 항목:
1. Database connection
2. User CRUD operations (Create, Read, Update)
3. UserPreference CRUD operations
4. CollectedArticle CRUD operations (filtering, sorting)
5. SentDigest CRUD operations
6. Feedback CRUD operations
7. Relationship testing (User ↔ Preference, User ↔ Digests, User ↔ Feedbacks, Article ↔ Feedbacks)

모든 CRUD 작업과 관계가 정상적으로 작동합니다!